In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torchvision.transforms import v2 as transformsV2
from torch.utils.data import DataLoader
from Dataset import MyDataset
from tqdm import tqdm
from Network import MyNetwork
import os
from torch.utils.tensorboard.writer import SummaryWriter
import itertools
import random

%load_ext autoreload
%autoreload 2

os.environ['KMP_DUPLICATE_LIB_OK']='True' # To prevent the kernel from dying

In [2]:
MEAN = np.array([0.5750, 0.6065, 0.6459])
STD = np.array([0.1854, 0.1748, 0.1794])
ROOT_DIRTECTORY = "c:\\Users\\jacop\\Desktop\\BSc\\Code\\WindTurbineImagesCategorization\\Data\\Dataset"
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Device in use: {DEVICE}")
hyper_parameters = {
    "input channels": 3,
    "number of classes": 5,
    "split": {"train": 0.6, "val": 0.2, "test": 0.2},
    "batch size": 32,
    "number of workers": 0,
    "epochs": 30,
    "epsilon": 1e-08,
    "weight decay": 1e-08,
}

Device in use: cuda:0


In [3]:
logs_dir = 'runs'
os.makedirs(logs_dir, exist_ok=True)
num_of_runs = len(os.listdir(logs_dir))
run_dir = os.path.join(logs_dir, f'run_')

In [4]:
def create_tqdm_bar(iterable, desc):
    return tqdm(enumerate(iterable), total=len(iterable), ncols=150, desc=desc)

In [5]:
transform = transformsV2.Compose([
    transformsV2.Resize((224, 224)),
    transformsV2.RandomHorizontalFlip(p=0.5),
    transformsV2.RandomVerticalFlip(p=0.5),
    transformsV2.RandomAdjustSharpness(sharpness_factor=2, p=0.5),
    transformsV2.RandomAutocontrast(p=0.5),
    transformsV2.RandomRotation(degrees=[0, 90]),
    transformsV2.ColorJitter(brightness=0.25, saturation=0.20),
    # Replace deprecated ToTensor()
    transformsV2.ToImage(),
    transformsV2.ToDtype(torch.float32, scale=True),
    transformsV2.Normalize(mean=MEAN.tolist(), std=STD.tolist()),
])

In [6]:
# def train_classifier(classifier, train_loader, val_loader, loss_func, tb_logger, epochs=10, name="default"):
def train_net(model, loss_function, device, dataloader_train, optimizer, hyper_parameters, logger, scheduler, name="default"):
    epochs = hyper_parameters["epochs"]

    validation_loss = 0
    for epoch in range(epochs):  # loop over the dataset multiple times

        """    Train step for one batch of data    """
        training_loop = create_tqdm_bar(
            dataloader_train, desc=f'Training Epoch [{epoch+1}/{epochs}]')

        training_loss = 0
        for train_iteration, batch in training_loop:
            model.train()  # Set the model to training mode
            optimizer.zero_grad()  # Reset the parameter gradients for the current minibatch iteration

            images, labels = batch
            labels -= 1  # Change the labels to start from 0
            labels = labels.type(torch.LongTensor)

            labels = labels.to(device)
            images = images.to(device)

            # Forward pass, backward pass and optimizer step
            predicted_labels = model(images)
            loss_train = loss_function(predicted_labels, labels)
            loss_train.backward()
            optimizer.step()

            # Accumulate the loss and calculate the accuracy of predictions
            training_loss += loss_train.item()

            # Running train accuracy
            _, predicted = predicted_labels.max(1)
            num_correct = (predicted == labels).sum()
            train_accuracy = float(num_correct)/float(images.shape[0])

            training_loop.set_postfix(train_loss="{:.8f}".format(
                training_loss / (train_iteration + 1)), val_loss="{:.8f}".format(validation_loss))
            # logger.add_scalar(f'{name}/train_loss', loss_train.item(),
            #                   (epoch * len(dataloader_train)) + train_iteration)

        logger.add_scalar(f'{name}/train_loss',
                          training_loss / len(dataloader_train), epoch)
        logger.add_scalar(f'{name}/train_accuracy', train_accuracy, epoch)

        scheduler.step()
        print(scheduler.get_last_lr())

In [7]:
def validate_model(model, loss_function, device, dataloader_validation):
    model.eval()  # Set model to evaluation mode
    total_loss = 0

    with torch.no_grad():
        for _, batch in enumerate(dataloader_validation):
            images, labels = batch
            labels -= 1  # Change the labels to start from 0
            labels = labels.type(torch.LongTensor)

            images = images.to(device)
            labels = labels.to(device)

            # Forward pass
            output = model(images)
            loss = loss_function(output, labels)
            total_loss += loss.item()

    return total_loss / len(dataloader_validation)

In [8]:
# Function to create all combinations of hyperparameters
def create_combinations(hyperparameter_grid):
    keys, values = zip(*hyperparameter_grid.items())
    return [dict(zip(keys, v)) for v in itertools.product(*values)]

# Function to randomly sample hyperparameters


def sample_hyperparameters(hyperparameter_grid, num_samples):
    samples = []
    for _ in range(num_samples):
        sample = {}
        for key, values in hyperparameter_grid.items():
            sample[key] = random.choice(values)
        samples.append(sample)
    return samples

In [9]:
# Define your hyperparameter grid
hyperparameter_grid = {
    'learning rate': [0.0002, 0.0004, 0.0006],
    'beta1': [0.9],
    'beta2': [0.999],
    'step size': [10],
    'gamma': [0.8, 0.9],
}

In [10]:
def hyperparameter_search(loss_function, device, dataloader_train, dataloader_validation, hyperparameter_grid, missing_hp):
    # Initialize with a large value for minimization problems
    best_performance = float('inf')
    best_hyperparameters = None
    run_counter = 0
    for hyper_parameters in hyperparameter_grid:
        # Empty memory before start
        if torch.cuda.is_available():
            torch.cuda.empty_cache()

        print(hyper_parameters)
        hyper_parameters.update(missing_hp)
        # Initialize model, optimizer, scheduler, logger

        model = MyNetwork(hyper_parameters)
        model.weight_initialization()
        model.to(DEVICE)

        optimizer = optim.Adam(model.parameters(),
                               lr=hyper_parameters["learning rate"],
                               betas=(hyper_parameters["beta1"],
                                      hyper_parameters["beta2"]),
                               weight_decay=hyper_parameters["weight decay"],
                               eps=hyper_parameters["epsilon"])

        scheduler = torch.optim.lr_scheduler.StepLR(
            optimizer, step_size=hyper_parameters["step size"], gamma=hyper_parameters["gamma"])

        logger = SummaryWriter(run_dir+"V4_"+str(run_counter),
                               comment=f'LR_{hyper_parameters["learning rate"]}')

        # Train the model
        train_net(model, loss_function, device, dataloader_train,
                  optimizer, hyper_parameters, logger, scheduler, name="MyNetV4")

        run_counter += 1
        # Evaluate model performance using validation set
        # Here, we're using validation loss as the performance metric
        validation_loss = validate_model(
            model, loss_function, device, dataloader_validation)

        # Update best hyperparameters if the current model has better performance
        if validation_loss < best_performance:
            best_performance = validation_loss
            best_hyperparameters = hyper_parameters

        logger.close()

    return best_hyperparameters

In [11]:
# Create Datasets and Dataloaders
dataset_train = MyDataset(root_directory=ROOT_DIRTECTORY, mode="train",
                          transform=transform, split=hyper_parameters["split"])
print(f"Created a new Dataset for training of length: {len(dataset_train)}")
dataset_validation = MyDataset(root_directory=ROOT_DIRTECTORY,
                               mode="val", transform=None, split=hyper_parameters["split"])
print(f"Created a new Dataset for validation of length: {
      len(dataset_validation)}")
dataloader_train = DataLoader(
    dataset_train, batch_size=hyper_parameters["batch size"], shuffle=True, num_workers=hyper_parameters["number of workers"], drop_last=False)
print(f"Created a new Dataloader for training with batch size: {
      hyper_parameters["batch size"]}")
dataloader_validation = DataLoader(dataset_validation, batch_size=hyper_parameters["batch size"], shuffle=True,
                                   num_workers=hyper_parameters["number of workers"], drop_last=False)
print(f"Created a new Dataloader for validation with batch size: {
    hyper_parameters["batch size"]}")

loss_function = nn.CrossEntropyLoss()


# Perform hyperparameter search
all_combinations = create_combinations(hyperparameter_grid)
# random_samples = sample_hyperparameters(hyperparameter_grid, 10)
best_hp = hyperparameter_search(loss_function, DEVICE, dataloader_train,
                                dataloader_validation, all_combinations, hyper_parameters)

Created a new Dataset for training of length: 1320
Created a new Dataset for validation of length: 440
Created a new Dataloader for training with batch size: 32
Created a new Dataloader for validation with batch size: 32
{'learning rate': 0.0002, 'beta1': 0.9, 'beta2': 0.999, 'step size': 10, 'gamma': 0.8}


Training Epoch [1/30]: 100%|█████████████████████████████████████████████| 42/42 [00:21<00:00,  2.00it/s, train_loss=13.89489201, val_loss=0.00000000]


[0.0002]


Training Epoch [2/30]: 100%|██████████████████████████████████████████████| 42/42 [00:20<00:00,  2.03it/s, train_loss=4.42253762, val_loss=0.00000000]


[0.0002]


...

[0.000486]


Training Epoch [29/30]: 100%|█████████████████████████████████████████████| 42/42 [00:20<00:00,  2.00it/s, train_loss=1.50473022, val_loss=0.00000000]


[0.000486]


Training Epoch [30/30]: 100%|█████████████████████████████████████████████| 42/42 [00:20<00:00,  2.02it/s, train_loss=1.52461299, val_loss=0.00000000]


[0.0004374]


In [12]:
print(f"Best hyperparameters: {best_hp}")
# {'learning rate': 0.0001, 'beta1': 0.9, 'beta2': 0.999, 'step size': 10, 'gamma': 0.9, 'input channels': 3, 'number of classes': 5, 'split': {'train': 0.6, 'val': 0.2, 'test': 0.2}, 'batch size': 32, 'number of workers': 0, 'epochs': 30, 'epsilon': 1e-08, 'weight decay': 1e-08}

# {'learning rate': 0.0001, 'beta1': 0.9, 'beta2': 0.999, 'step size': 10, 'gamma': 0.8, 'input channels': 3, 'number of classes': 5, 'split': {'train': 0.6, 'val': 0.2, 'test': 0.2}, 'batch size': 32, 'number of workers': 0, 'epochs': 30, 'epsilon': 1e-08, 'weight decay': 1e-08

Best hyperparameters: {'learning rate': 0.0002, 'beta1': 0.9, 'beta2': 0.999, 'step size': 10, 'gamma': 0.8, 'input channels': 3, 'number of classes': 5, 'split': {'train': 0.6, 'val': 0.2, 'test': 0.2}, 'batch size': 32, 'number of workers': 0, 'epochs': 30, 'epsilon': 1e-08, 'weight decay': 1e-08}


In [13]:
# {'learning rate': 0.0002, 'beta1': 0.9,
#     'beta2': 0.999, 'step size': 10, 'gamma': 0.8}